In [1]:
import os
import csv
from countries_languages import country_to_language

import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime

from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name



/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/AssemblyHelpers.py:54: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


# LOADING DATASETS

In [2]:
def remove_accents(text):
    return unidecode(text)


def get_country_code(country):
    matching_rows = countries_codes.loc[countries_codes['Country'] == country, ' Code']
    return matching_rows.iloc[0] if not matching_rows.empty else None

In [ ]:
#USING TERRELLS DATASET
# leagues_value = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/Fixed TransferMarkt Market Values - MarketValues.csv')
# leagues_value['Accents Removed'] = leagues_value['Name'].apply(lambda x: unidecode(x))
# leagues_value['Team 1 Code'] = leagues_value['Nationality'].apply(get_country_code)
# leagues_value.to_csv("Most Updated Edited Transfermarkt Dataset.csv", index=False)



In [3]:
#PLAYER DATA - MARKET VALUE AND SALARY
leagues_salary = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/latest_capology_data_money_fixed.csv')

leagues_value = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/Most Updated Edited Transfermarkt Dataset.csv')


leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/most_updated_transfermarkt_dataset.csv')

#COUNTRY CODES
countries_codes = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/countries_and_codes.csv')

#FIFA RANKINGS
fifa_rankings = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/fifa_ranking-2023-10-26 - fifa_ranking-2023-07-20.csv')

#MATCH DATA - LINEUPS AND RESULTS
lineups_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/newest_lineups_data_march_24.csv')
results_data = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/newest_results_data_march_24.csv')

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_51958/1138293197.py:7: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  leagues_value_large = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/most_updated_transfermarkt_dataset.csv')


GETTING NULL NAMES - SALARIES & VALUES

In [4]:
nullnames_salaries = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/nullnames_salaries.csv')
nullnames_values = pd.read_csv('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/nullnames_values.csv')
# nullnames_salaries = AssemblyHelpers.get_names_with_conditions_salary(leagues_salary) #result_names_null

# nullnames_values = AssemblyHelpers.get_names_with_conditions_values(leagues_value) #result_names_null_values


In [ ]:
#SCRATCH 
# Assuming result_names_null_values and result_names_null are lists of data
# result_names_null_values = pd.DataFrame(result_names_null_values)
# result_names_null = pd.DataFrame(result_names_null)

# # Now you can save them as CSVs
# result_names_null_values.to_csv("MarketValues_null_names.csv", index=False)
# result_names_null.to_csv("Salaries_null_names.csv", index=False)

# nullnames_salaries = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/Salaries_null_names.csv')
# nullnames_salaries = list(nullnames_salaries['Name'])

# nullnames_values = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/MarketValues_null_names.csv')
# nullnames_values = list(nullnames_values['Name'])


# Helper Methods

### new version of find $ info from player with extra logic added

In [5]:
#HELPER METHODS
##SELENIUM WRAPPER FUNCTION 

def useSeleniumToFindMoney(
        NAME_TO_FIND, #the name it is looking up
        NATIONALITY_INPUT_PLAYER, #natl_test
        DATE_INPUT_MATCH, #input_year_test
        SALARY_BOOLEAN):
    if(SALARY_BOOLEAN == False):
        print('finding salary with selenium')
        marketvalue = AssemblyHelpers.seleniumLookUpValueWrapperFunction(NAME_TO_FIND, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH)
        return marketvalue
    elif(SALARY_BOOLEAN == True):
        salary = AssemblyHelpers.capology_selenium_lookup(NAME_TO_FIND, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH)
        return salary

    else:
        return 0



#MATCH DATE FILTER 
def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''



#FINDING MONEY INFO USSING PLAYER NAME
def tryToFindPlayerMoney(
        DATABASE_RETURN_MESSAGE, #what the DB said when trying to find money info
        NAME_TO_FIND, #the name it found in the DB (name_match[0] originally)
        NATIONALITY_INPUT_PLAYER, #natl_test
        DATE_INPUT_MATCH, #input_year_test
        SALARY_BOOLEAN, #IF TRUE, SALARIES. OTHERWISE VALUES
        ORIGINAL_SEARCH_NAME #name_match[2]
        ):

    if((DATABASE_RETURN_MESSAGE == "not in DB any of 3 seasons.") or (DATABASE_RETURN_MESSAGE == "asswipe was in DB in before or after season")):
        print('newwwnewnenwenewnwenwenwew', DATABASE_RETURN_MESSAGE)
        
        if DATABASE_RETURN_MESSAGE == 'asswipe was in DB in before or after season':
            money_thisplayer = [0, 'Was in  DB before or after season', True, True]
            return money_thisplayer

        
        
        

        INTERNET_LOOKUP_RESULTING_NAMES = AssemblyHelpers.false_name_match_lookup(ORIGINAL_SEARCH_NAME, NATIONALITY_INPUT_PLAYER)
        NUM_NAMES_REMAINING = len(list(INTERNET_LOOKUP_RESULTING_NAMES))
        print(INTERNET_LOOKUP_RESULTING_NAMES)
        #1 MATCH
        if(NUM_NAMES_REMAINING == 1):
            #FIND INFO USING SELENIUM
            # NAME_FOR_SELENIUM_SEARCH = next(iter(INTERNET_LOOKUP_RESULTING_NAMES))
            # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH, SALARY_BOOLEAN)
            # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
            money_thisplayer = [0, 'Lookup Required', True, False]
            return money_thisplayer
        #MULTIPLE MATCHES
        elif(NUM_NAMES_REMAINING >= 2):
            print('goes to FUDOM')
            #FILTER THE NAMES USING THE DATE OF THE MATCH
            list_to_filter_with_match_date = list(INTERNET_LOOKUP_RESULTING_NAMES)
            #NEW FUNCTION 3 
            money_thisplayer, name_results = filter_using_date_of_match(list_to_filter_with_match_date, NATIONALITY_INPUT_PLAYER, DATE_INPUT_MATCH, SALARY_BOOLEAN) 
            return money_thisplayer
        #0 MATCHES
        elif(NUM_NAMES_REMAINING == 0):
            #This is return case 9
            money_thisplayer = [0, 'Online Lookup Required', True, False] 
            return money_thisplayer
    
    else:
        money_thisplayer = [0, 'Online Lookup Required', True, False]
        return money_thisplayer
    


    #old
    # elif(DATABASE_RETURN_MESSAGE == "was in DB in before or after season"):
    #     0==0
    

##OVERARCHING Controller FUNCTION 
def new_overarching(starter_number, name_input, nationality_input, date_input, salary_boolean, competition_input, combination_dict_matches, combination_dict_NON_matches, combination_dict_multi_matches):
    """
    Inputs: Name, Date, Nationality, Salary Boolean 
        salary_boolean
            true – salary
            false – Market Value

    Outputs: Money_thisplayer, name_match
    Outputs to add: error, error type, error msg
    """

    #SETTING INITIAL VALUES
    example_problem = name_input
    natl_test = nationality_input
    input_year_test = date_input

    if 'U-20' in natl_test:
        print('u-20 team')
        natl_test = natl_test.split(' U-20')[0]

    corrrect_code = countries_codes[countries_codes['Country'] == natl_test][' Code'].unique()[0]
    

    if salary_boolean == True:
        0==0
        #dataset_nationality = leagues_salary[leagues_salary['Team 1 Code'] == corrrect_code]['Name'].unique()
    else:
        dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == corrrect_code]['Name'].unique()

    money_thisplayer = [0, 'not done yet', True, False]
    secondmatch_required = False

    #FIRST TRY LOOKING FOR NAME
    name_match = AssemblyHelpers.lookup_name(example_problem, natl_test, input_year_test, salary_boolean, competition_input)
    
    match_case = name_match[1]
    original_db_name_for_search = name_match[2]
    print('match is', name_match)

    

    if(match_case == 'single'):
        NAME_MATCH_FIRST_TRY = name_match[0]
        #print(NAME_MATCH_FIRST_TRY)
        money_thisplayer = AssemblyHelpers.find_money_info_from_name(NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values)

        money_found = money_thisplayer[0]
        search_db_return_case = money_thisplayer[1]
        online_lookup_required = money_thisplayer[2]

        if ((money_found == 0) & (online_lookup_required == True)):         #call the find_money_info_extended once you write it. inputs are = (NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, name_match[2])
            #NEW FUNCTION 1
            if '.' in input_year_test:
                yearstr = input_year_test.split(".")[2]
                full_num = int('20' + yearstr)
            elif '-' in input_year_test:
                if len(input_year_test.split("-")[2]) == 4:
                    yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                    full_num = int(yearstr)
                elif len(input_year_test.split("-")[2]) == 2:
                    if len(input_year_test.split("-")[0]) == 4:
                        yearstr = input_year_test.split("-")[0]
                        full_num = int(yearstr)
                    else:
                        yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                        full_num = int('20' + yearstr)
                else:
                    print('broke on year parsing', input_year_test.split("-"))
                # elif len(input_year_test.split("-")[0]) == 4:
                    # yearstr = input_year_test.split("-")[0]
                    # full_num = yearstr
            
            #print(f'singi trying {NAME_MATCH_FIRST_TRY} and {int('20' + yearstr)}')
            player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num)].reset_index()
            
            if len(player_bigvalue_dataset) >= 1:
                #print(player_bigvalue_dataset.at[0, 'Market Value'])
                #money_thisplayer[0] =
                #print('working youre good')
                money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                        #money_thisplayer[2] = True
                        money_thisplayer[1] = 'Large Dataset had 0'
            else:
                if len(leagues_value_large[((leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num - 1)) | 
                                       ((leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num+ 1))]) >= 2:
                    money_thisplayer = [0, 'Not in either dataset for currect season but present in both seasons around match', False, True]
                    # leagues_value_large[((leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num - 1)) | 
                    #                    ((leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value_large['Season'] == full_num + 1))]['Market Value'].mean()
                else:
                    if ((leagues_value[leagues_value['Name'] == NAME_MATCH_FIRST_TRY]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == NAME_MATCH_FIRST_TRY]['Season'].max() == (full_num - 1))):
                        money_last_season = leagues_value[(leagues_value['Name'] == NAME_MATCH_FIRST_TRY) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                        money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                    else:
                        money_thisplayer = tryToFindPlayerMoney(search_db_return_case, NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
            
        elif(search_db_return_case == 'working'):
            0==0
            #return to the end basically
        elif(search_db_return_case == 'working - imputed'):
            0==0
            #when ur done this should be removed. go into the function find_money_info and remove case
                    
    elif(match_case == 'multiple'):
        print('multi match')

        if '.' in input_year_test:
            yearstr = input_year_test.split(".")[2]
            full_num = int('20' + yearstr)
        elif '-' in input_year_test:
            if len(input_year_test.split("-")[2]) == 4:
                yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                full_num = int(yearstr)
            elif len(input_year_test.split("-")[2]) == 2:
                if len(input_year_test.split("-")[0]) == 4:
                    yearstr = input_year_test.split("-")[0]
                    full_num = int(yearstr)
                else:
                    yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                    full_num = int('20' + yearstr)

        combination_key = (example_problem, full_num)

        if combination_key in combination_dict_matches:

            print('combo is in matches dict')
            #correct_name = combination_dict_matches[combination_key]
            #print('correct name is ', correct_name)

            SINGLE_NAME_MATCH_CORRECT_SEASON = combination_dict_matches[combination_key]
                
            money_thisplayer = AssemblyHelpers.find_money_info_from_name(SINGLE_NAME_MATCH_CORRECT_SEASON[0], natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 

            money_found = money_thisplayer[0]
            search_db_return_case = money_thisplayer[1]
            online_lookup_required = money_thisplayer[2]

            if ((money_found == 0) & (online_lookup_required == True)):  
                #NEW FUNCTION 1     
                if '.' in input_year_test:
                    yearstr = input_year_test.split(".")[2]
                    full_num = int('20' + yearstr)
                elif '-' in input_year_test:
                    if len(input_year_test.split("-")[2]) == 4:
                        yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                        full_num = int(yearstr)
                    elif len(input_year_test.split("-")[2]) == 2:
                        if len(input_year_test.split("-")[0]) == 4:
                            yearstr = input_year_test.split("-")[0]
                            full_num = int(yearstr)
                        else:
                            yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                            full_num = int('20' + yearstr)
                    else:
                        print('broke on year parsing', input_year_test.split("-"))
        
                #print(f'multiii trying {SINGLE_NAME_MATCH_CORRECT_SEASON} and {int('20' + yearstr)}')
                if type(SINGLE_NAME_MATCH_CORRECT_SEASON) == list:
                    SINGLE_NAME_MATCH_CORRECT_SEASON = SINGLE_NAME_MATCH_CORRECT_SEASON[0]
                player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value_large['Season'] == full_num)].reset_index()
                
                if len(player_bigvalue_dataset) >= 1:
                    #print(player_bigvalue_dataset.at[0, 'Market Value'])
                    #money_thisplayer[0] =
                    #print('working youre good')
                    money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                    if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                        #money_thisplayer[2] = True
                        money_thisplayer[1] = 'Large Dataset said 0'
                else:
                    if ((leagues_value[leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1))):
                        money_last_season = leagues_value[(leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                        money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                    else:
                        money_thisplayer = tryToFindPlayerMoney(search_db_return_case, SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                        
            elif(search_db_return_case == 'working'):
                0==0
                #return to the end basically
            elif(search_db_return_case == 'working - imputed'):
                0==0
                #when ur done this should be removed. go into the function find_money_info and remove case
            
            #in this situation, you need to do find money info from name. 
            #use the name that is in the dict 
        elif combination_key in combination_dict_NON_matches:
            print('combination in non-matches dict')#if you find it in the non matches dictionary
            #save the list of names it returned 
            #then return money thisplayer 
            NAMES_MATCHES_LIST_CORRECT_SEASON = combination_dict_NON_matches[combination_key]
            money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, ORIGINAL NAME WAS {example_problem}', True, False]
        elif combination_key in combination_dict_multi_matches:
            #stop the loop here 
            filtered_names_match_date = combination_dict_multi_matches[combination_key]
            money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {filtered_names_match_date}', True, False]
        else:
            print('combo is not in matches, multi-matches or non-matches dict')
        
            NAMES_MATCHES_LIST = name_match[0]
            
            
            #FILTER THE NAMES USING THE YEAR OF THE MATCH
            NAMES_MATCHES_LIST_CORRECT_SEASON = AssemblyHelpers.filter_candidates_using_year(starter_number, NAMES_MATCHES_LIST, natl_test, input_year_test, salary_boolean)
            

            NUMBER_OF_NAMES_MATCHING = len(NAMES_MATCHES_LIST_CORRECT_SEASON)
            #print(f'left is {NUMBER_OF_NAMES_MATCHING}')
            
            if(NUMBER_OF_NAMES_MATCHING == 1): #YELLOW 2
                
                
                SINGLE_NAME_MATCH_CORRECT_SEASON = NAMES_MATCHES_LIST_CORRECT_SEASON[0]
                
                money_thisplayer = AssemblyHelpers.find_money_info_from_name(SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 

                money_found = money_thisplayer[0]
                search_db_return_case = money_thisplayer[1]
                online_lookup_required = money_thisplayer[2]

                if ((money_found == 0) & (online_lookup_required == True)):  
                    #NEW FUNCTION 1     
                    if '.' in input_year_test:
                        yearstr = input_year_test.split(".")[2]
                        full_num = int('20' + yearstr)
                    elif '-' in input_year_test:
                        if len(input_year_test.split("-")[2]) == 4:
                            yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                            full_num = int(yearstr)
                        elif len(input_year_test.split("-")[2]) == 2:
                            if len(input_year_test.split("-")[0]) == 4:
                                yearstr = input_year_test.split("-")[0]
                                full_num = int(yearstr)
                            else:
                                yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                                full_num = int('20' + yearstr)
                        else:
                            print('broke on year parsing', input_year_test.split("-"))
            
                    #print(f'multiii trying {SINGLE_NAME_MATCH_CORRECT_SEASON} and {int('20' + yearstr)}')
                    print(full_num, SINGLE_NAME_MATCH_CORRECT_SEASON)
                    player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value_large['Season'] == full_num)].reset_index()
                    
                    if len(player_bigvalue_dataset) >= 1:
                        #print(player_bigvalue_dataset.at[0, 'Market Value'])
                        #money_thisplayer[0] =
                        #print('working youre good')
                        money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                        if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                            #money_thisplayer[2] = True
                            money_thisplayer[1] = 'Large Dataset said 0'
                    else:
                        if ((leagues_value[leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1))):
                            money_last_season = leagues_value[(leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                            money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                        else:
                            money_thisplayer = tryToFindPlayerMoney(search_db_return_case, SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                            
                elif(search_db_return_case == 'working'):
                    0==0
                    #return to the end basically
                elif(search_db_return_case == 'working - imputed'):
                    0==0
                    #when ur done this should be removed. go into the function find_money_info and remove case
                
            elif(NUMBER_OF_NAMES_MATCHING >= 2):
                #use match date to filter 
                #print('list of names is ', NAMES_MATCHES_LIST_CORRECT_SEASON)
                #print('we are here', NUMBER_OF_NAMES_MATCHING)
                if('Olympics' in competition_input):
                    money_thisplayer = [0, F'Olympics row. Couldn\'t filter names using Transfermarkt. remaining matches were {NAMES_MATCHES_LIST_CORRECT_SEASON}', True, False]
                    #the filtering is not going to work 
                else:
                
                    filtered_names_match_date = AssemblyHelpers.multiNameMatchDateLookup(NAMES_MATCHES_LIST_CORRECT_SEASON, natl_test, input_year_test) 

                    #1 MATCH
                    if(type(filtered_names_match_date) == str): #YELLOW 3
                        
                        if '.' in input_year_test:
                            yearstr = input_year_test.split(".")[2]
                            full_num = int('20' + yearstr)
                        elif '-' in input_year_test:
                            if len(input_year_test.split("-")[2]) == 4:
                                yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                                full_num = int(yearstr)
                            elif len(input_year_test.split("-")[2]) == 2:
                                if len(input_year_test.split("-")[0]) == 4:
                                    yearstr = input_year_test.split("-")[0]
                                    full_num = int(yearstr)
                                else:
                                    yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                                    full_num = int('20' + yearstr)
                        #print('we here', filtered_names_match_date, full_num, example_problem)
                        combination_key = (example_problem, full_num)
                        combination_dict_matches[combination_key] = [filtered_names_match_date]
                        print(f'added key {combination_key} to dict. value is {filtered_names_match_date}')
                        #at this point, example problem and full_num become a combination 
                        #that combination should be a key, and the value should be filtered_names_match_date
                        #add a new key+value combination to the dictionary here. 

                        NAME_MATCH_SINGLE = filtered_names_match_date

                        #name_match[0] = NAME_MATCH_SINGLE
                        #print('we hereeeee', NAME_MATCH_SINGLE)
                        money_thisplayer = AssemblyHelpers.find_money_info_from_name(NAME_MATCH_SINGLE, natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 
                        
                        money_found = money_thisplayer[0]
                        search_db_return_case = money_thisplayer[1]
                        online_lookup_required = money_thisplayer[2]

                        if ((money_found == 0) & (online_lookup_required == True)):         #call the find_money_info_extended once you write it. inputs are = (NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, name_match[2])
                            # NAME_FOR_SELENIUM_SEARCH = NAME_MATCH_SINGLE
                            # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, natl_test, input_year_test, salary_boolean)
                            # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {salary_boolean}', True, False]

                            if len(leagues_value_large[(leagues_value_large['Name'] == NAME_MATCH_SINGLE) & (leagues_value_large['Season'] == full_num)]) >= 1:
                                money_thatseason_large_db = leagues_value_large[(leagues_value_large['Name'] == NAME_MATCH_SINGLE) & (leagues_value_large['Season'] == full_num)]['Market Value'].max()
                                money_thisplayer = [money_thatseason_large_db, 'Used Large Dataset', False, False]

                            elif ((leagues_value[leagues_value['Name'] == NAME_MATCH_SINGLE]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == NAME_MATCH_SINGLE]['Season'].max() == (full_num - 1))):
                                money_last_season = leagues_value[(leagues_value['Name'] == NAME_MATCH_SINGLE) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                                money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                            else:
                                #money_thisplayer = tryToFindPlayerMoney(search_db_return_case, NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                                money_thisplayer = [0, 'Lookup Required', True, False]
                            #return money_thisplayer
                            
                        elif(search_db_return_case == 'working'):
                            0==0
                            #return to the end basically
                        elif(search_db_return_case == 'working - imputed'):
                            0==0
                            #when ur done this should be removed. go into the function find_money_info and remove case


                    #MULTIPLE MATCHES
                    elif(len(filtered_names_match_date) >= 2): #RED 6

                        print('added a row to multi matches dictionary.')
                        combination_key_multi_match = (example_problem, full_num)
                        combination_dict_multi_matches[combination_key_multi_match] = [filtered_names_match_date]
                        
                        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {filtered_names_match_date}', True, False]
                        #NNNNEW - add to the dictionary here 
                        #return money_thisplayer, name_match
                    #0 MATCHES
                    elif(len(filtered_names_match_date) == 0): #RED 5
                        #print('we are here. bahrain case')
                        if '.' in input_year_test:
                            yearstr = input_year_test.split(".")[2]
                            full_num = int('20' + yearstr)
                        elif '-' in input_year_test:
                            if len(input_year_test.split("-")[2]) == 4:
                                yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                                full_num = int(yearstr)
                            elif len(input_year_test.split("-")[2]) == 2:
                                if len(input_year_test.split("-")[0]) == 4:
                                    yearstr = input_year_test.split("-")[0]
                                    full_num = int(yearstr)
                                else:
                                    yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                                    full_num = int('20' + yearstr)

                        combination_key_nomatch = (example_problem, full_num)
                        combination_dict_NON_matches[combination_key_nomatch] = [NAMES_MATCHES_LIST_CORRECT_SEASON]
                        print(f'added a row to non matches dict. player was {example_problem} in {full_num}.')
                        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, ORIGINAL NAME WAS {example_problem}', True, False]
                        #return money_thisplayer, name_match
                    
            elif(NUMBER_OF_NAMES_MATCHING == 0):#RETURN CASE 4

                print('here rc4')
                
                money_thisplayer, name_results = filter_using_date_of_match(NAMES_MATCHES_LIST, natl_test, input_year_test, salary_boolean) 
                if money_thisplayer == [0, 'Lookup Required', True, False]:
                    SINGLE_NAME_MATCH_CORRECT_SEASON = name_results
                    combination_key = (example_problem, full_num)
                    combination_dict_matches[combination_key] = [SINGLE_NAME_MATCH_CORRECT_SEASON]
                    print(f'added key {combination_key} to dict. value is {SINGLE_NAME_MATCH_CORRECT_SEASON}')
                    player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value_large['Season'] == full_num)].reset_index()
                    
                    if len(player_bigvalue_dataset) >= 1:
                        #print(player_bigvalue_dataset.at[0, 'Market Value'])
                        #money_thisplayer[0] =
                        #print('working youre good')
                        money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                        if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                            #money_thisplayer[2] = True
                            money_thisplayer[1] = 'Large Dataset said 0'
                    else:
                        if ((leagues_value[leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1))):
                            money_last_season = leagues_value[(leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                            money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                        else:
                            money_thisplayer = tryToFindPlayerMoney(money_thisplayer[1], SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                
    

    elif(match_case == 'none'):
        #secondmatch_required = True
        #print('ya 1')

        NAME_TO_SEARCH = name_match[2]

        NAME_FOURTH_TRY = AssemblyHelpers.fourth_try_name_search(NAME_TO_SEARCH, natl_test, salary_boolean)

        MATCH_CASE_FOURTH_TRY = NAME_FOURTH_TRY[0]

        if(MATCH_CASE_FOURTH_TRY == 'single'): #YELLOW 1
            NAME_FOUND = NAME_FOURTH_TRY[1]
            money_thisplayer = AssemblyHelpers.find_money_info_from_name(NAME_FOUND, natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 

            money_found = money_thisplayer[0]
            search_db_return_case = money_thisplayer[1]
            online_lookup_required = money_thisplayer[2]

            if '.' in input_year_test:
                yearstr = input_year_test.split(".")[2]
                full_num = int('20' + yearstr)
            elif '-' in input_year_test:
                if len(input_year_test.split("-")[2]) == 4:
                    yearstr = input_year_test.split("-")[2] #this is a 4 digit number
                    full_num = int(yearstr)
                elif len(input_year_test.split("-")[2]) == 2:
                    if len(input_year_test.split("-")[0]) == 4:
                        yearstr = input_year_test.split("-")[0]
                        full_num = int(yearstr)
                    else:
                        yearstr = input_year_test.split("-")[2] #this is a 2 digit number
                        full_num = int('20' + yearstr)

            if ((money_found == 0) & (online_lookup_required == True)): #call the find_money_info_extended once you write it. inputs are = (NAME_MATCH_FIRST_TRY, natl_test, input_year_test, salary_boolean, name_match[2])
                if ((leagues_value[leagues_value['Name'] == NAME_FOUND]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == NAME_FOUND]['Season'].max() == (full_num - 1))):
                    money_last_season = leagues_value[(leagues_value['Name'] == NAME_FOUND) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                    money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                else:
                    money_thisplayer = tryToFindPlayerMoney(search_db_return_case, NAME_FOUND, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
                
            elif(search_db_return_case == 'working'):
                0==0
                #return to the end basically
            elif(search_db_return_case == 'working - imputed'):
                0==0
                #when ur done this should be removed. go into the function find_money_info and remove case

        elif(MATCH_CASE_FOURTH_TRY == 'multiple'):#RED 3

            print('here 4th try')
    
            LIST_TO_FILTER = NAME_FOURTH_TRY[1]
            money_thisplayer, name_results = filter_using_date_of_match(LIST_TO_FILTER, natl_test, input_year_test, salary_boolean) 
            if money_thisplayer == [0, 'Lookup Required', True, False]:
                SINGLE_NAME_MATCH_CORRECT_SEASON = name_results
                combination_key = (example_problem, full_num)
                combination_dict_matches[combination_key] = [SINGLE_NAME_MATCH_CORRECT_SEASON]
                print(f'added key {combination_key} to dict. value is {SINGLE_NAME_MATCH_CORRECT_SEASON}')
                player_bigvalue_dataset = leagues_value_large[(leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value_large['Season'] == full_num)].reset_index()
                
                if len(player_bigvalue_dataset) >= 1:
                    #print(player_bigvalue_dataset.at[0, 'Market Value'])
                    #money_thisplayer[0] =
                    #print('working youre good')
                    money_thisplayer = [player_bigvalue_dataset.at[0, 'Market Value'], 'Used Large Dataset', False, False]
                    if player_bigvalue_dataset.at[0, 'Market Value'] == 0:
                        #money_thisplayer[2] = True
                        money_thisplayer[1] = 'Large Dataset said 0'
                else:
                    if ((leagues_value[leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1)) & (leagues_value_large[leagues_value_large['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON]['Season'].max() == (full_num - 1))):
                        money_last_season = leagues_value[(leagues_value['Name'] == SINGLE_NAME_MATCH_CORRECT_SEASON) & (leagues_value['Season'] == (full_num - 1))]['Market Value'].max()
                        money_thisplayer = [money_last_season, f'Data Ended before season : {full_num}', False, False]
                    else:
                        money_thisplayer = tryToFindPlayerMoney(money_thisplayer[1], SINGLE_NAME_MATCH_CORRECT_SEASON, natl_test, input_year_test, salary_boolean, original_db_name_for_search)
        
        elif(MATCH_CASE_FOURTH_TRY == 'none'): #SELENIUM
            #print('ya 2')
            original_db_name_for_search = name_match[2]

            accents_removed_name = remove_accents(original_db_name_for_search)
            #print(accents_removed_name)
            equal_names = [name for name in dataset_nationality if name == accents_removed_name]
            #(accents_removed_name in dataset_nationality)

            if equal_names:
                0==0
            else:
            #print
                matching_names = [name for name in dataset_nationality if name.endswith(accents_removed_name)]
                #print('weere here', matching_names)
                if len(matching_names) >= 2:
                    0==0 #more than 1 person remains
                elif len(matching_names) == 1:
                    0==0 #single matching name
                    #print('heylo')
                    money_thisplayer = AssemblyHelpers.find_money_info_from_name(matching_names[0], natl_test, input_year_test, salary_boolean, nullnames_salaries, nullnames_values) 
                else:
                    0==0 #no matches 
                    #print('nerr')
                    # MONEY_FOUND_ONLINE_0 = useSeleniumToFindMoney(original_db_name_for_search, natl_test, input_year_test, salary_boolean)
                    # money_thisplayer = [MONEY_FOUND_ONLINE_0, f'Online WAS Lookup Required - salary boolean is {salary_boolean}', True, False]
                    money_thisplayer = [0, 'Lookup Required', True, False]
    else:
        0==0 #RED 7

    return money_thisplayer, name_match



# Working

In [6]:
#Changing countries_codes - click 1 time at start
data = {'Country': ['British Virgin Islands'], ' Code': ['BVI'], 'Accents Removed': ['British Virgin Islands']}
one_row_dataframe = pd.DataFrame(data)
countries_codes = pd.concat([countries_codes, one_row_dataframe], ignore_index=True)

data = {'Country': ['Tahiti'], ' Code': ['TI'], 'Accents Removed': ['Tahiti']}
one_row_dataframe = pd.DataFrame(data)
countries_codes = pd.concat([countries_codes, one_row_dataframe], ignore_index=True)

countries_codes['Accents Removed'] = countries_codes['Country'].apply(remove_accents)


In [36]:
lookup_dataset_1 = pd.DataFrame()


In [8]:
combination_dict_matches = {}
combination_dict_multi_matches = {}

combination_dict_NON_matches = {}


In [37]:
pd.set_option('display.max_columns', None)
#Dataset names
# leagues_salary - the Player dataset for salaries 
# leagues_value - 
# countries_codes #might be of use on new CONCACAF and CONMEBOL case - near matches 
# fifa_rankings -
# lineups_data
# results_data
# nullnames_salaries
# nullnames_values

#ENGLAND CASE - 1240 - DONE
#CZECH CASE CELUSTA - NEAR THERE - DONE


#HONDURAS - 8240, 8241 - done

#IRaq - 2626, DONE

#belgium thing - 1640, 1641 excel row is 1649 - DONE

#medel is excel 334 - code is 329, 330 – DONE
#second row medel is 363 in excel – DONE

#Row of Comoros guys - 3940, 3941 - DONE
#in these cases, there are guys who have non-zero MV in the season after the game. but just resolve these on the 2nd go round 

#fiji assholes - 2654, 2655 - DONE 

#c accent thing - 1241, 1242 - DONE
#c accent 2nd time - 1218, 1219 - DONE

#olympics thing (iraq) - 2633, 2634 - DONE

#NULL GUY CASE - EXCEL ROW IS 4641 , Code is 4613-4614 - DONE

#cyrillic - 1205, 1206 - DONE (FOR NOW)


#0, 312 - THE u-20 WORLD CUP 

#312, 400 - done
#400, 500 - done 


#500, 600 

#521 - ivanovic thing 
#after the match date filter returns one guy, add that to a dictionary 

salary_boolean = False
for k in range(1200, 1300): 
    nationality_this = lineups_data.at[k, 'TeamName']
    competition_this = lineups_data.at[k, 'Competition']
    #nationality_this = lineups_data.at[k, 'Match']
    date_this = lineups_data.at[k, 'Date']
    country_code_this = lineups_data.at[k, ' Code']
    print(nationality_this, date_this)
    for i in range(1, 12): #1, 12 # HAZARD IS 10, 11 #MEDEL 358 is 6, 7 #medel 339 is 7, 8
        
        column = lineups_data.columns[i]
        player_this = lineups_data.at[k, f'{column}']
        if player_this == "5. Keane W.":
            player_this = "5. Keane M." 
        elif player_this == '1. Tomáš':
            player_this = '1. T. Vaclik'
        elif player_this == '10. Hazard':
            brothers_belge = False
            for j in range(1, 11):
                player_this_belge = lineups_data.at[k, f'{lineups_data.columns[j]}']
                if 'Hazard T' in player_this_belge:
                    print(' both hazard brothers played here')
                    player_this = '10. Hazard E.'
                    brothers_belge = True
                    break
                    
        # Your logic here, replace the print statement with your function
        #print(player_this)
        if '.' in date_this:
            yearstr = date_this.split(".")[2]
            full_num = int('20' + yearstr)
        elif '-' in date_this:
            if len(date_this.split("-")[2]) == 4:
                yearstr = date_this.split("-")[2] #this is a 4 digit number
                full_num = int(yearstr)
            elif len(date_this.split("-")[2]) == 2:
                if len(date_this.split("-")[0]) == 4:
                    yearstr = date_this.split("-")[0]
                    full_num = int(yearstr)
                else:
                    yearstr = date_this.split("-")[2] #this is a 2 digit number
                    full_num = int('20' + yearstr)
        if(player_this) == '-':
            #print('nthing here')
            data_row = {
                'Match ID': lineups_data.at[k, 'Match ID'],
                'Competition': competition_this,
                'Match': lineups_data.at[k, 'Match'],
                'Date': date_this,
                'Nationality': nationality_this,
                'Team Country Code': country_code_this,
                'Season': full_num,
                'Status': 'FAIL',
                'Name': '-', #this will be zero if a fail happens 
                'Market Value': 0,
                'Lookup Still Required?': False,
                'Lookup Return Case': 'Null Name',
                'Impute Required?': False,
                'Name(s) Found': '',
                'ORIGINAL JERSEY': '-',
                'Match Case': 'none' #single, multiple, none or cyrillic
            }

            lookup_dataset_1 = pd.concat([lookup_dataset_1, pd.DataFrame([data_row])], ignore_index=True)
        elif pd.isna(player_this):
            #print('nullski here')
            data_row = {
                'Match ID': lineups_data.at[k, 'Match ID'],
                'Competition': competition_this,
                'Match': lineups_data.at[k, 'Match'],
                'Date': date_this,
                'Nationality': nationality_this,
                'Team Country Code': country_code_this,
                'Season': full_num,
                'Status': 'FAIL',
                'Name': 'Null', #this will be zero if a fail happens 
                'Market Value': 0,
                'Lookup Still Required?': False,
                'Lookup Return Case': 'Null Name',
                'Impute Required?': False,
                'Name(s) Found': '',
                'ORIGINAL JERSEY': '-',
                'Match Case': 'none' #single, multiple, none or cyrillic
            }

            lookup_dataset_1 = pd.concat([lookup_dataset_1, pd.DataFrame([data_row])], ignore_index=True)
        else:
            thisrow_info = (new_overarching(i, player_this, nationality_this, date_this, salary_boolean, competition_this, combination_dict_matches, combination_dict_NON_matches, combination_dict_multi_matches))
            print(thisrow_info)
            if((thisrow_info[0][2] == True) & ((thisrow_info[0][0] == 0)| (thisrow_info[0][0] == 0.0))):
                #print(thisrow_info)
                0==0
                print(player_this, '*********FAIL', thisrow_info)
                status_var = 'FAIL'
            else:
                0==0 
                print(player_this, 'success', thisrow_info[0][0])
                status_var = 'SUCCESS'
            data_row = {
                'Match ID': lineups_data.at[k, 'Match ID'],
                'Competition': competition_this,
                'Match': lineups_data.at[k, 'Match'],
                'Date': date_this,
                'Nationality': nationality_this,
                'Team Country Code': country_code_this,
                'Season': full_num,
                'Status': status_var,
                'Name': f"{thisrow_info[1][0]}", #this will be zero if a fail happens 
                'Market Value': thisrow_info[0][0],
                'Lookup Still Required?': thisrow_info[0][2],
                'Lookup Return Case': thisrow_info[0][1],
                'Impute Required?': thisrow_info[0][3],
                'Name(s) Found': thisrow_info[1][0],
                'ORIGINAL JERSEY': thisrow_info[1][2],
                'Match Case': thisrow_info[1][1] #single, multiple, none or cyrillic

            }

            lookup_dataset_1 = pd.concat([lookup_dataset_1, pd.DataFrame([data_row])], ignore_index=True)
            

            #this row match ID - DONE
            #this row competition - DONE
            #this row year - DONE
            #player's nationality - DONE
            #whether it worked - DONE
            #player name - DONE
            #market value from money_thisplayer - DONE
            #whether lookup is required, money this player [2] - DONE
            #the return case, money this player [1] - DONE 
                
            #impute required, the last part of money this player - DONE
            #set of names found, name match[0]
            #the original jersey, name_match[2]
            

            #player's player ID - find this if its a success  
            
  

Denmark 04.09.15
1. Шмейхель
ride
match is (0, 'cyrillic', '1. Шмейхель')
([0, 'not done yet', True, False], (0, 'cyrillic', '1. Шмейхель'))
1. Шмейхель *********FAIL ([0, 'not done yet', True, False], (0, 'cyrillic', '1. Шмейхель'))
3. Кьяер
ride
match is (0, 'cyrillic', '3. Кьяер')
([0, 'not done yet', True, False], (0, 'cyrillic', '3. Кьяер'))
3. Кьяер *********FAIL ([0, 'not done yet', True, False], (0, 'cyrillic', '3. Кьяер'))
4. Аггер
ride
match is (0, 'cyrillic', '4. Аггер')
([0, 'not done yet', True, False], (0, 'cyrillic', '4. Аггер'))
4. Аггер *********FAIL ([0, 'not done yet', True, False], (0, 'cyrillic', '4. Аггер'))
5. Дурмиси
ride
match is (0, 'cyrillic', '5. Дурмиси')
([0, 'not done yet', True, False], (0, 'cyrillic', '5. Дурмиси'))
5. Дурмиси *********FAIL ([0, 'not done yet', True, False], (0, 'cyrillic', '5. Дурмиси'))
6. Якобсен
ride
match is (0, 'cyrillic', '6. Якобсен')
([0, 'not done yet', True, False], (0, 'cyrillic', '6. Якобсен'))
6. Якобсен *********FAIL ([0,

In [38]:
for k in range(1200, 1300):
    print(k, lineups_data.at[k, 'TeamName'], lineups_data.at[k, 'Date'])

1200 Denmark 04.09.15
1201 Albania 04.09.15
1202 Serbia 04.09.15
1203 Armenia 04.09.15
1204 Armenia 07.09.15
1205 Denmark 07.09.15
1206 Albania 07.09.15
1207 Portugal 07.09.15
1208 Albania 08.10.15
1209 Serbia 08.10.15
1210 Portugal 08.10.15
1211 Denmark 08.10.15
1212 Armenia 11.10.15
1213 Albania 11.10.15
1214 Serbia 11.10.15
1215 Portugal 11.10.15
1216 Georgia 12.11.20
1217 North Macedonia 12.11.20
1218 Serbia 12.11.20
1219 Scotland 12.11.20
1220 Northern Ireland 12.11.20
1221 Slovakia 12.11.20
1222 Hungary 12.11.20
1223 Iceland 12.11.20
1224 Georgia 08.10.20
1225 Belarus 08.10.20
1226 North Macedonia 08.10.20
1227 Kosovo 08.10.20
1228 Scotland 08.10.20
1229 Israel 08.10.20
1230 Norway 08.10.20
1231 Serbia 08.10.20
1232 Bosnia-Herzegovina 08.10.20
1233 Northern Ireland 08.10.20
1234 Slovakia 08.10.20
1235 Ireland 08.10.20
1236 Iceland 08.10.20
1237 Romania 08.10.20
1238 Bulgaria 08.10.20
1239 Hungary 08.10.20
1240 Bulgaria 22.03.19
1241 Montenegro 22.03.19
1242 England 22.03.19
1243 

In [42]:
lookup_dataset_1#[lookup_dataset_1['Name'] == 'Sekou']

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case
0,605,"Euros 2016, Qualifiers",Denmark vs Albania,04.09.15,Denmark,DK,2015,FAIL,0,0.0,True,not done yet,False,0,1. Шмейхель,cyrillic
1,605,"Euros 2016, Qualifiers",Denmark vs Albania,04.09.15,Denmark,DK,2015,FAIL,0,0.0,True,not done yet,False,0,3. Кьяер,cyrillic
2,605,"Euros 2016, Qualifiers",Denmark vs Albania,04.09.15,Denmark,DK,2015,FAIL,0,0.0,True,not done yet,False,0,4. Аггер,cyrillic
3,605,"Euros 2016, Qualifiers",Denmark vs Albania,04.09.15,Denmark,DK,2015,FAIL,0,0.0,True,not done yet,False,0,5. Дурмиси,cyrillic
4,605,"Euros 2016, Qualifiers",Denmark vs Albania,04.09.15,Denmark,DK,2015,FAIL,0,0.0,True,not done yet,False,0,6. Якобсен,cyrillic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,655,"Euros 2020, Qualifiers",Serbia vs Portugal,07.09.19,Portugal,PT,2019,SUCCESS,Bernardo Silva,80000000.0,False,working,False,Bernardo Silva,Bernardo Silva,single
1096,655,"Euros 2020, Qualifiers",Serbia vs Portugal,07.09.19,Portugal,PT,2019,SUCCESS,Bruno Fernandes,48000000.0,False,working,False,Bruno Fernandes,B Fernandes,single
1097,655,"Euros 2020, Qualifiers",Serbia vs Portugal,07.09.19,Portugal,PT,2019,SUCCESS,Danilo Pereira,24000000.0,False,working,False,Danilo Pereira,Danilo Pereira,single
1098,655,"Euros 2020, Qualifiers",Serbia vs Portugal,07.09.19,Portugal,PT,2019,SUCCESS,Cristiano Ronaldo,60000000.0,False,working,False,Cristiano Ronaldo,Cristiano Ronaldo,single


In [40]:
big_lookup_dataset = lookup_dataset_1
#big_lookup_dataset = pd.concat([big_lookup_dataset, lookup_dataset_1], ignore_index=True)

In [41]:
big_lookup_dataset.to_csv('lookup_dataset_1_rows_1200_1300.csv')

In [9]:
#testing cyrillic idea
import requests

url = "https://www.typingbaba.com/translator/russian-to-english-translation.php"
response = requests.get(url)

if response.status_code == 200:
    print("Request successful")
    print("Response content:", response.text)
else:
    print("Error:", response.status_code)


Request successful
Response content: <!DOCTYPE html>
<html lang="en">
<head><meta http-equiv="Content-Type" content="text/html; charset=utf-8">
      <title>Russian To English Translation | Online Converter</title>
    <meta name="viewport" content="width=device-width" />	
      <link rel="canonical" href="https://www.typingbaba.com/translator/russian-to-english-translation.php"/>

<meta name="google-site-verification" content="AuhcNOhclNTe494-tVLfUUL5_c8cetye9WO-iiRJF1A" />
 
<meta name="copyrighted-site-verification" content="53a1d8d5cbdfdc27" />
	<meta name="google-site-verification" content="M8A88SDqYnv6e8N7FIxmTlgz3qVamraqMsIwGFkrx7k" />
	<meta name="msvalidate.01" content="55C723FF3FDA1D251647D0590EA4FC20" />
	<link rel="shortcut icon" href="https://www.typingbaba.com/img/ficon.png">
<meta property="og:image" content="https://www.typingbaba.com/img/fb.png" />
<meta name="w3dir" content="da7d3da0fa774ec0d7eec7bbe4f699304a888b7059bd798faa78a2eebe0d685b">
<script>var screen_width=Ma

## Decompose name lookups

In [5]:
### OLD - TEST Names for adding accents
#"4 I. Perišić"
#21 G. Kļuškins	
#"5 O. Laizāns"
#7 A. Višņakovs
#20 E. Hallfreðsson
#22 J. Böðvarsson
#"7 A. Žulpa"
#"18 J. Bořil"
#8 E. Vaitkūnas
#"17 M. Mandžukić"
#"3 B. Bereszyński"
#"18 O. Selnæs"
#"20 Ł. Piszczek"
#1 W. Szczęsny
#10 Thiago Alcântara
#"3 A. Jędrzejczyk"
#"5 K. Mączyński"
#"3 A. Toșca"
#"16 S. Plătică"
#"8 João Moutinho"
#"13 P. Hrošovský"
#"6 Wataru Endō"
#"14 Andrei Rațiu"
#"10. Çalhanoğlu"
#"15. O'Dowda"
#"O`Donnell S.G." THIS GUY HAS 2 FIRST INITIALS . 

#"19. Brown J-b."
#"#20. Hudson-Odoi"
#"A. Oxlade-Chamberlain"
#17. Miller M-n
#2. Garcia J-C.
#"3. Băluţă T.C."

#hyphen cases


#input_string = "7 D Maria"
#natltest = 'Scotland'

input_string = "11. Sterling R."


#then this belongs at the start of the word 
#so remove it from the end of the word 

#AFTER ADDING TRAILING INITIALS 



AssemblyHelpers.cyrillic_to_latin(input_string)

#CYRILLIC TESTS

#"13. Джон Д.К."



'11. Sterling R.'

In [7]:
### OLD - refactoring process_String_newest

#REMOVE JERSEY NUM STRING
cleaned_string = re.sub(r'^\d{1,2}[. ]', '', input_string)
tokens = cleaned_string.split()
final_string = ""


#ADD STARTING INITIAL 
if len(tokens) >= 1 and not re.match(r'^[A-Za-zÀ-ÖØ-öø-ÿŁ]{2,}$', tokens[0]):
    initial_match = re.match(r'^([A-Za-zÀ-ÖØ-öø-ÿŁ]+\.)+$|[A-Za-zÀ-ÖØ-öø-ÿŁ]\.$|[A-Za-zÀ-ÖØ-öø-ÿŁ]$', tokens[0])
    if initial_match:
        final_string += initial_match.group()


#CREATE MAIN PHRASE - REQUIRED. 
main_phrase = " ".join(word for word in tokens if len("".join(char for char in word if char.isalpha())) >= 2)
if main_phrase:
    final_string += " " + main_phrase


#ADD END INITIAL 
if len(tokens) >= 2 and not re.match(r'^[A-Za-zÀ-ÖØ-öø-ÿ]{2,}$', tokens[1]):
    end_initial_match = re.match(r'^([A-Za-zÀ-ÖØ-öø-ÿ]+\.)+$|[A-Za-zÀ-ÖØ-öø-ÿ]\.$|[A-Za-zÀ-ÖØ-öø-ÿ]$', tokens[1])
    if end_initial_match:
        final_string = "".join(char for char in end_initial_match.group() if char.isalpha()) + " " + final_string


#ACCOUNT FOR IF LAST INITIAL LONGER THAN 1 LETTER
if len(tokens[-1]) >= 3 and tokens[-1].endswith("."):
    trailing_token_is_long_starting_initial = True
    final_string = final_string.replace(tokens[-1], "")
    final_string = final_string[:-1]

#ACCOUNT FOR HYPHENATED LAST INITIAL OR HYPHENATED NAME

if '-' in tokens[-1]:
    parts = tokens[-1].split('-')
    if len(parts[0]) >= 2 and len(parts[1]) >= 2:
        0==0
    else:
        final_string = tokens[-1] + ' ' + final_string
        tokens_hyphen_case = final_string.split()
        if(tokens_hyphen_case[0] == tokens_hyphen_case[-1]):
            final_string = final_string[:-len(tokens_hyphen_case[-1])].rstrip()
            print(final_string)

#HYPHENATeD First INITIAL OR HYPHENATED NAME
if '-' in tokens[0]:
    parts = tokens[0].split('-')
    if len(parts[0]) >= 2 and len(parts[1]) >= 2:
        0==0
    else:
        final_string = tokens[0] + ' ' + final_string

                                            # Check if the final phrase ends in a period
                                            # if final_string.endswith("."):
                                            #     # Extract the last word, remove the period, and move it to the start of final_string
                                            #     last_word = final_string.split()[-1].rstrip('.')
                                            #     final_string = last_word + " " + final_string

                                            #     # Remove the last word from the end of the string
                                            #     final_string = ' '.join(final_string.split()[:-1])

                                            #     final_string = final_string.strip()

# Separate the final string by " " and remove non-alphabet characters for each token
final_tokens = [re.sub(r'[^A-Za-zÀ-ÖØ-öø-ÿřäáāņļćčšðŽž\'`ūńæŁęúâąșăãýōţțğÇçü-]', '', token) for token in final_string.split()]

                                            # If the first two tokens are the same, remove one token
                                            # if len(final_tokens) >= 2 and final_tokens[0] == final_tokens[1]:
                                            #     final_tokens.pop(0)

if len(final_tokens[0]) >= 2 and final_tokens[0].isupper():
    final_tokens[0] = final_tokens[0][0]


joined_string = " ".join(final_tokens)


joined_string, final_tokens

('R Sterling', ['R', 'Sterling'])

In [14]:
#Testing find closest string
dataset_nationality = leagues_value[(leagues_value['Nationality'] == 'Argentina')]['Name'].unique()
#(leagues_value['Nationality'] == f'Republic of {natltest}')

result = AssemblyHelpers.find_closest_string_newEST(joined_string, dataset_nationality, final_tokens, input_string)

NameError: name 'joined_string' is not defined

#  Debugging

## Overarching Function

In [ ]:
#TEST CASES

##WORK (with WORKING CASES)

#Random other cases
# example_problem = "Saleh Al Shehri" "Fahad Al Muwallad"  "Abdullah Otayf" "Salem Al Dawsari"
# natl_test = 'Saudi Arabia'
# input_year_test = "05.06.21"

In [15]:
search_name, final_tokens_name = AssemblyHelpers.process_string_newest_ii(example_problem, this_competition)
final_tokens_name

added leading initial 
added main phrase  Maguire
added end initial  Maguire
last initial longer than 1 letter  Maguire
hyphenated last initial  Maguire
hyphenated first initial  Maguire
process accents, tokens: ['Maguire']
process multiple initials, tokens:  ['Maguire']


['Maguire']

In [22]:
#6. Maguire
#11. Sterling
#20. Hudson-Odoi    
example_problem = '6. Maguire'
natl_test = 'England'
input_year_test = '22.03.19'
this_competition = 'Euros 2020, Qualifiers'

salary_boolean = False
dataset_nationality = leagues_value[leagues_value['Nationality'] == 'England']['Name'].unique()


# search_name = 'Maguire'
# final_tokens_name = ['Maguire']

AssemblyHelpers.find_closest_string_newEST(search_name, dataset_nationality, final_tokens_name, example_problem)

['Harry Maguire']

In [ ]:
first, second = new_overarching(example_problem, natl_test, input_year_test, salary_boolean)

In [ ]:
first, second
#AssemblyHelpers.lookup_name(example_problem, natl_test, input_year_test, salary_boolean)

In [ ]:
#WORKING TEST CASES

# example_problem = "Ahmed Abdul Wahid" 
# natl_test = 'Yemen'
# input_year_test = "05.06.21"

# example_problem = "6. J. Seri"
# natl_test = 'Cote d\'Ivoire'
# input_year_test = '17.11.15'

# example_problem = '19 B. Traoré'
# natl_test = 'Burkina Faso'
# input_year_test = "07.10.16"

# example_problem = '6. Lovren'
# natl_test = 'Croatia'
# input_year_test = '08.06.19'

# example_problem = "17 A. Baba"
# natl_test = "Ghana" 
# input_year_test = "07.10.16"

# example_problem = "17. N'Diaye M."
# natl_test = 'Mali'
# input_year_test = '17.01.17'

#CASE WHERE THERE WERE POINTS ABOVE LAST X AXIS TICKMARK
# example_problem = "Salman Al Faraj"
# natl_test = 'Saudi Arabia'
# input_year_test = "05.06.21"

#HAD ISSUES WITH THE GRAPH
# example_problem = "Mohammed Boqshan"
# natl_test = 'Yemen'
# input_year_test = "05.06.21"

# example_problem = "9. Zelarayan"
# natl_test = 'Armenia'
# input_year_test = "08.10.21"

#REQUIRES THAT EXTRA SEARCH WITH ALL THE LINKS
# example_problem = "5. Ala Addin Mahdi"
# natl_test = 'Yemen'
# input_year_test = "05.06.21"

#MARKET VALUE GRAPH IS NULL CASE #2
# example_problem = "Kum-Chol Ri"
# natl_test = 'Korea DPR'
# input_year_test = "13.10.2015"

#MARKET VALUE GRAPH IS NULL CASE #3
# example_problem = "Saddam Hussain"
# natl_test = 'Pakistan'
# input_year_test = "12.03.2015"

#GAVE A MANAGER URL ON FIRST LOOK
# example_problem = "S. Djeparov"
# natl_test = 'Uzbekistan'
# input_year_test = "03-09-2015"

#MARKET VALUE OVER 365 DAYS OUT CASE
# example_problem = "Mahmoud Abu Warda"
# natl_test = 'Palestine'
# input_year_test = "14-11-2019"

#CASE WHERE YOU NEEDED DIV[2] NOT DIV[3] TO FIND ELEMENTS
# example_problem = "Mudir Al Radaei"
# natl_test = 'Yemen'
# input_year_test = "05.06.21" # he also played "24-03-2016"

##Had a null transfermarkt page, had to pursue Lovren case solution where looked for names. 
##Actually found none. impute required from here
# example_problem = "Husain Ali Pele"
# natl_test = 'Bahrain'
# input_year_test = "08.10.15"

# example_problem = "Manaf Saeed" #"Emad Mansoor"
# natl_test = 'Yemen'
# input_year_test = "05.06.21"

In [ ]:
#CASES TO REVISIT

##SEE NOTES IN NOTION 
# example_problem = "Hesham Nayem"
# natl_test = 'Bahrain'
# input_year_test = "24.03.16"


# OLD TEST CASES

In [ ]:
# OLD CASES


# example_problem = "6. J. Seri"
# natl_test = 'Cote d\'Ivoire'
# input_year_test = '17.11.15'

# example_problem = '19 B. Traoré'
# natl_test = 'Burkina Faso'
# input_year_test = "07.10.16"

# example_problem = '6. Lovren'
# natl_test = 'Croatia'
# input_year_test = '08.06.19'

# example_problem = "17 A. Baba"
# natl_test = "Ghana" 
# input_year_test = "07.10.16"

# example_problem = "17. N'Diaye M."
# natl_test = 'Mali'
# input_year_test = '17.01.17'

#CASE WHERE THERE WERE POINTS ABOVE LAST X AXIS TICKMARK
# example_problem = "Salman Al Faraj"
# natl_test = 'Saudi Arabia'
# input_year_test = "05.06.21"

#HAD ISSUES WITH THE GRAPH
# example_problem = "Mohammed Boqshan"
# natl_test = 'Yemen'
# input_year_test = "05.06.21"

# example_problem = "9. Zelarayan"
# natl_test = 'Armenia'
# input_year_test = "08.10.21"

#REQUIRES THAT EXTRA SEARCH WITH ALL THE LINKS
# example_problem = "5. Ala Addin Mahdi"
# natl_test = 'Yemen'
# input_year_test = "05.06.21"

##Had a null transfermarkt page, had to pursue Lovren case solution where looked for names. 
##Actually found none. impute required from here
# example_problem = "Husain Ali Pele"
# natl_test = 'Bahrain'
# input_year_test = "08.10.15"

#MARKET VALUE GRAPH IS NULL CASE #1
# example_problem = "Hesham Nayem"
# natl_test = 'Bahrain'
# input_year_test = "24.03.16"

#MARKET VALUE GRAPH IS NULL CASE #2
# example_problem = "Kum-Chol Ri"
# natl_test = 'Korea DPR'
# input_year_test = "13.10.2015"

#MARKET VALUE GRAPH IS NULL CASE #3
# example_problem = "Saddam Hussain"
# natl_test = 'Pakistan'
# input_year_test = "12.03.2015"

#GAVE A MANAGER URL ON FIRST LOOK
# example_problem = "S. Djeparov"
# natl_test = 'Uzbekistan'
# input_year_test = "03-09-2015"

#MARKET VALUE OVER 365 DAYS OUT CASE
# example_problem = "Mahmoud Abu Warda"
# natl_test = 'Palestine'
# input_year_test = "14-11-2019"


#CASE WHERE YOU NEEDED DIV[2] NOT DIV[3] TO FIND ELEMENTS
# example_problem = "Mudir Al Radaei"
# natl_test = 'Yemen'
# input_year_test = "24-03-2016" # he also played "05.06.21" 

#MARKET VALUE OUT OF RANGE OF GAME (over 365 days away)
# example_problem = "Mudir Al Radaei"
# natl_test = 'Yemen'
# input_year_test = "05.06.21" # he played on "05.06.21" #and 24-03-2016



#Random other cases
# example_problem = "Saleh Al Shehri" "Fahad Al Muwallad"  "Abdullah Otayf" "Salem Al Dawsari"
# natl_test = 'Saudi Arabia'
# input_year_test = "05.06.21"

In [ ]:
#MORE TEST CASES 

#WORKING - 1 CALL TO FIND CLOSEST STRING
# example_problem = "10. Mane Sa."
# natl_test = 'Senegal'
# input_year_test = "19.06.18"

#WORKING - 1 CALL TO FIND CLOSEST STRING
# example_problem = "9. Son Heungmin"
# natl_test = 'South Korea'
# input_year_test = '17.01.17'

#WORKING - 1 CALL TO FIND CLOSEST STRING
# example_problem = "13. Koo Jacheol"
# natl_test = 'South Korea'
# input_year_test = '26.06.14'

#WORKING - 1 CALL TO FIND CLOSEST STRING
# example_problem = "17. Sarr P."
# natl_test = 'Senegal'
# input_year_test = "04.12.22"

#WORKING - 1 CALL TO FIND CLOSEST STRING
# example_problem = '4. Torres Pau'
# natl_test = 'Spain'
# input_year_test = "14.11.21"

#WORKING - 1 CALL TO FIND CLOSEST STRING
# example_problem = "6 Sane S."
# natl_test = 'Senegal'
# input_year_test = '19.06.18'

#WORKING - 1 CALL TO FIND CLOSEST STRING
# example_problem = '11 A Kolarov'
# natl_test = 'Serbia'
# input_year_test = '15.10.13'

#WORKING - 1 CALL TO FIND CLOSEST STRING
# example_problem = "22. Wague M." 
# natl_test = "Senegal"
# input_year_test = "24.06.18"

#WORKING - 1 CALL TO FIND CLOSEST STRING
# example_problem = "17. Badou Ndiaye"
# natl_test = "Senegal"
# input_year_test = "28.01.17"

#WORKING - 1 CALL TO FIND CLOSEST STRING
# example_problem = "8. Sylla Y." 
# natl_test = 'Mali'
# input_year_test = "17.01.17"

#WORKING - 1 CALL TO FIND CLOSEST STRING
# example_problem = "6. Krejčí L. II."
# natl_test = 'Czech Republic'
# input_year_test = '02.06.22'

#WORKING - 2 CALLs TO FIND CLOSEST STRING (accent case)
# example_problem = "6. Saiss"
# natl_test = 'Morocco'
# input_year_test = "30.01.22"


#WORKING - 3 CALLS FIND CLOSEST STRING (backtick case)
# example_problem = "9. Eto`o"
# natl_test = 'Cameroon'
# input_year_test = '13.06.14'


#WORKING - 1 CALL TO FIND CLOSEST STRING
# example_problem = "13. A. N`Diaye"
# natl_test = 'Senegal'
# input_year_test = "14.11.2017"


###WORKING - 3 CALLS TO FIND CLOSEST STRING, 1 MORE TO NEW FUNCTION
# example_problem = "17 A. Baba"
# natl_test = "Ghana" 
# input_year_test = "07.10.16"